## Lecture III How is agricultural production increasing?



We&rsquo;ve seen that growth in food production is typically greater than
population growth.  But where is this growth coming from?

Food (and crops in particular) are the classical example of
production.  The &ldquo;classical&rdquo; economists in the 18th century, when
most income came from agriculture, thought there were three main
&ldquo;factors&rdquo; of production:

-   Land
-   Labor
-   Capital

Thus one might write the technical relationship between &ldquo;factors&rdquo; (or
inputs) and output as
$$
    \text{Crop output} = F(\mbox{Land},\mbox{Labor},\mbox{Capital}).
 $$



### Functional forms



We have good reason to think that $F$ displays constant returns to
scale; i.e., is homogeneous of degree one.  Write it as
$F(x_1,x_2,\dots,x_n)$ (thus abstracting from the classical factors
of production).

Observationally, it&rsquo;s also often the case the cost share for
different factors of production remains constant, even when prices
change.  If we combine these facts (linearly homogeneous, constant
cost shares) with an assumption that farmers operating this
production function are profit maximizing price-takers and a
technical assumption that $F$ is continuously differentiable, then
one can prove that $F$ is &ldquo;Cobb-Douglas&rdquo;, or
$$
      F(x_1,x_2,\dots,x_n) = A\prod_{i=1}^nx_i^{\alpha_i},
  $$
where $\sum_{i=1}^n\alpha_i=1$.  This is a result first established
by the economist Paul Douglas and the mathematician Charles Cobb in 1928.

If we observe output at time $t$ for country $j$, say $y^j_t$ and inputs $x^j_{it}$, we can take the
logarithm of the Cobb-Douglas production function, obtaining
$$
     \log y^j_t = \log A^j_t + \sum_{i=1}^n\alpha^j_i\log(x^j_{it}).
  $$
Note that we&rsquo;ve allowed the cost-share parameters $\alpha$ to vary
across both inputs and also countries, but *not* over time.



### Total Factor Productivity



The term $A$ is sometimes called &ldquo;Total Factor Productivity&rdquo; (TFP),
because increases in $A$ increase productivity of all factors.  If
we take differences in log output over time we get
$$
      \Delta\log y^j_t = \Delta\log A^j_t + \sum_{i=1}^n\alpha^j_i\Delta\log(x^j_{it}).
   $$
Recall that changes in logs approximate percent changes or growth
rates, so we can use this equation to decompose output growth into
growth in input use and TFP growth.



### Data on Food Production



What’s happened to food production over recent decades?
See
[https://www.ers.usda.gov/data-products/international-agricultural-productivity/](https://www.ers.usda.gov/data-products/international-agricultural-productivity/).

Data on TFP, output, factor use, and factor shares can be found at
[https://docs.google.com/spreadsheets/d/1J_Yoo2eBgABBy8Hnvh2Vf7GYowELZBYTVwt-3NtjUsc/](https://docs.google.com/spreadsheets/d/1J_Yoo2eBgABBy8Hnvh2Vf7GYowELZBYTVwt-3NtjUsc/),
which is publicly readable.



### Reading Sheets



I&rsquo;ve written a python package `eep153_tools` which includes tools to
handle authentication as well as reading google sheets as pandas
DataFrames.  First we have to deal with authentication, by decrypting
credentials to access particular files (you should only have to do
this part once):



In [4]:
# %pip install eep153_tools
# %pip install python_gnupg

from eep153_tools.sheets import decrypt_credentials
decrypt_credentials('../students.json.gpg')

FileNotFoundError: [Errno 2] No such file or directory: '../students.json.gpg'

To check that this worked, the following gives a list of
emails for &ldquo;service<sub>accounts</sub>&rdquo; that now have credentials&#x2013;you can
then &ldquo;share&rdquo; google sheets with these.



In [1]:
!ls ~/.eep153.service_accounts/

With those credentials established



In [1]:
from eep153_tools.sheets import read_sheets

#### Read a bunch of google worksheets into a dictionary of dataframes
data = read_sheets('https://docs.google.com/spreadsheets/d/1J_Yoo2eBgABBy8Hnvh2Vf7GYowELZBYTVwt-3NtjUsc/edit#gid=1532023339')['Data']

data.keys()

This gives a dictionary of dataframes, but these dataframes need to be
tidied up some.  For example, look at `Outall_Index`:



In [1]:
data['Outall_Index'].head()

We really just want selected columns, and an index that depends on country and year:



In [1]:
import pandas as pd

Data = {'Level':'Level','Country':'Country/Territory','WDI Code':'ISO3','Year':'Year','Output':'Outall_Index','TFP':'TFP_Index','Land':'Land_Index','Labor':'Labor_Index','Capital':'Capital_Index','Materials':'Materials_Index'}


df = data.rename(columns={v:k for k,v in Data.items()})
df = df[Data.keys()].set_index(['WDI Code','Level','Country','Year'])

# Deal with some duplicate indices (drops regions)
df = df.loc[~df.index.duplicated(),:]
df

### Visualizing data on ag production



Plot growth in output for all countries for 1961 on:



In [2]:
import cufflinks as cf
cf.go_offline()

df['Output'].unstack().T.iplot(title="Index of Agricultural Output",
                               yTitle='Value of Output Index',
                               xTitle='Year')

NameError: name 'df' is not defined

Compare world growth in outputs, inputs, and TFP:



In [1]:
import numpy as np

world = df.xs('World',level='Level').replace(0,np.nan).dropna(how='any')

# Drop unnecessary index levels
world = world.droplevel(['WDI Code','Country'])

# Put in log differences
dworld = np.log(world).diff()
dworld['Inputs'] = dworld['Output'] - dworld['TFP']

dworld.mean()

NameError: name 'df' is not defined

And a graph of growth rates:



In [1]:
dworld[['Output','Inputs','TFP']].iplot(title="Growth rates of output, inputs, & TFP",
                                           xTitle="Year")

That&rsquo;s the overall picture for the world.  Now &ldquo;drill down&rdquo; and
consider just production in the US:



In [1]:
select = df.xs('USA',level='WDI Code').dropna(how='any')

# Drop unnecessary index levels
select = select.droplevel(['Level','Country'])

dselect = np.log(select).diff()
dselect['Inputs'] = dselect['Output'] - dselect['TFP']

dselect.mean()

And here a graph of growth in indices of inputs & outputs since 1961:



In [1]:
select = df.xs('USA',level='WDI Code').dropna(how='any').droplevel(['Level','Country'])

select = select/select.loc[1961,:]
select.iplot()

Compare with India, which in recent years has had a level of
agricultural output close to that of the US:



In [1]:
select = df.xs('IND',level='WDI Code').dropna(how='any').droplevel(['Level','Country'])

select = select/select.loc[1961,:]
select.iplot()